In [1]:
using Bloqade
using Distributions
using LegendrePolynomials
using PythonCall
using LinearAlgebra
using Printf

include("../src/BloqadeControl.jl")
using .BloqadeControl

In [2]:
# Specify parametrization of waveforms
n_sites = 2
basis = "legendre"
n_basis = 10
use_sigmoid_envolope = true
tf = 2.0
F0 = [(x)->0.,(x)->0.]

# Specify training parameters
params = ones((2,n_basis))
lr = 5e-2
w_l2 = 1e-3
n_iter = 500
method = "adam"

# Specify parameters of Rydberg atoms
distance = 5.95
atoms = generate_sites(ChainLattice(), n_sites, scale=distance);
local_detuning = [-1, 1]
global_detuning = 0.0
n_samples = 10

# Specify target state
target_state = uniform_state(n_sites) ## uniform superposition state
J = UniformScaling(1.)
tar_op = density_matrix(target_state).state
M = J - tar_op;
M = GeneralMatrixBlock(M; nlevel=2);

In [3]:
# Setup optimizer
generator = WaveformGenerator(n_sites, basis, n_basis, use_sigmoid_envolope, F0, tf)
model = BloqadeModel(atoms, local_detuning, global_detuning, n_samples, M);
optim = Optimizer(params, lr, w_l2, n_iter, method)

Optimizer([1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0], 0.05, 0.001, 500, "adam", [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])

In [4]:
# Finite difference method
dx = 1e-4 # stepsize
grad0 = grad_fdm(model, generator, optim, dx)
grad_fdm1 = vec(grad0)
n0 = norm(grad0)
@sprintf "Norm of grad (FDM) = %.4f" n0

"Norm of grad (FDM) = 3.2911"

In [5]:
# Our method
optim.params = params
update_waveform!(model, generator, optim.params)
backward!(model, generator, optim)
grad1 = copy(optim.g)
grad1 = vec(grad1)
n1 = norm(grad1)
overlap1 = dot(grad0./n0, grad1./n1)
@sprintf "No. of samples: %i, norm of grad (diff-analog) = %.4f, overlap with grad-fdm = %.4f" model.n_samples n1 overlap1

"No. of samples: 10, norm of grad (diff-analog) = 3.8481, overlap with grad-fdm = 0.9123"

In [6]:
# If we increase the number of samples
model.n_samples = 1000
optim.params = params
update_waveform!(model, generator, optim.params)
backward!(model, generator, optim)
grad2 = copy(optim.g)
grad2 = vec(grad2)
n2 = norm(grad2)
overlap2 = dot(grad0./n0, grad2./n2)
@sprintf "No. of samples: %i, norm of grad (diff-analog) = %.4f, overlap with grad-fdm = %.4f" model.n_samples n2 overlap2

"No. of samples: 1000, norm of grad (diff-analog) = 3.2970, overlap with grad-fdm = 0.9983"